In [1]:
import sys
sys.path.append("../src")

import warnings
warnings.filterwarnings("ignore")

from pprint import PrettyPrinter
# Configure pretty printer
pp = PrettyPrinter(compact=True)
print_ = lambda x: pp.pprint(x)

In [2]:
import json
import random
import pandas as pd
from pathlib import Path

from Utils.utils import PPT
from Config.config import PATHS
from Utils.interaction import Performer
from Utils.plot_utils import PlotVSMeasures
from Utils.utils import Grid

from Classes.cognitive_model_agents import AttendanceM2, FairnessM3, MFPM2, AvailableSpaceM2


## 2 Players

### Show AIC

In [ ]:
file_2PL = Path(PATHS['parameter_fit_results'], 'MLE/best_fit_2P.json')
df_2P = pd.read_json(file_2PL)
df_2P.index = df_2P.model_name
df_parameters_2P = df_2P[['model_name', 'free_parameters']]
df_2P.drop(columns=['model_name', 'free_parameters'], inplace=True)
df_2P.sort_values(by='model_name', inplace=True)
df_2P['deviance'] = df_2P['deviance'].apply(lambda x: int(x))
df_2P['AIC'] = df_2P['AIC'].apply(lambda x: int(x))
df_2P

In [ ]:
latex_string = df_2P.to_latex()
print(latex_string)

In [ ]:
from Utils.LaTeX_utils import PrintLaTeX

latex_string_parameters = ''

for model, row in df_parameters_2P.iterrows():
    if model in ['Attendance-M2', 'MFP-M2']:
        parameters = row['free_parameters']
        latex_string_parameters += r'\textbf\{' + model + '\}\n'
        latex_string_parameters += PrintLaTeX.print_parameters(parameters)

print(latex_string_parameters)

### Simulate best fit model

In [ ]:
with open(file_2PL, 'r') as f:
    params_dict = json.load(f)

def get_free_parameters(model_name, params_dict):
    model_names = params_dict['model_name']
    idx = model_names.index(model_name)
    return params_dict['free_parameters'][idx]

# BEST_MODEL = AttendanceM2
# BEST_MODEL = FairnessM3
BEST_MODEL = FairnessM3

fixed_parameters = {
	"threshold":0.5,
	"num_agents":2,
}
free_parameters = get_free_parameters(BEST_MODEL.name(), params_dict)
simulation_parameters = {
	'num_rounds':50,
	'verbose':False
}

In [ ]:
data_folder = PATHS['parameter_fit_results'] / Path('MLE')
num_episodes = 23 # same as human groups
random.seed(1)
seeds = [random.randint(0, 1000) for _ in range(num_episodes)]


Performer.simple_run(
    agent_class=BEST_MODEL,
    fixed_parameters=fixed_parameters,
    free_parameters=free_parameters,
    simulation_parameters=simulation_parameters,
    measures=[],
    data_folder=data_folder,
    seeds=seeds
)


### Comparing with human data

In [3]:
# Loading human data
file_name = '2-player-UR.csv'
data_folder = Path('..', 'data', 'human')
file = data_folder / Path(file_name)
print(f'Loading data from {file}...')
data_human = pd.read_csv(file)
data_human.drop(columns=['room', 'choice'], inplace=True)
data_human['model'] = 'human'

# Loading simulated data

BEST_MODEL = AvailableSpaceM2

data_folder = PATHS['parameter_fit_results'] / Path('MLE')
file = data_folder / Path(f'{BEST_MODEL.name()}.csv')
print(f'Loading data from {file}...')
data_sim = pd.read_csv(file)
data_sim['model'] = BEST_MODEL.name()

df = PPT.concat_dfs(data_human, data_sim)

p = PlotVSMeasures(df)

Loading data from ..\data\human\2-player-UR.csv...
Loading data from C:\Users\andra\Documents\Cognitive_Models_El_Farol_Bar_Problem\reports\MLE\AvailableSpace-M2.csv...


In [ ]:
image_file = PATHS['parameter_fit_results'] / Path('MLE', f'two_way_comparisons.png')
measure_pairs = [['efficiency', 'inequality'], ['efficiency', 'attendance']]
p.two_way_comparisons(measure_pairs, image_file)

In [ ]:
# measures=['efficiency', 'entropy']
# image_file = PATHS['parameter_fit_results'] / Path('MLE', f'{measures[0]}_vs_{measures[1]}.png')
# p.plot_vs(
#     pair_measures=measures,
#     file=image_file
# )
measures = ['efficiency', 'inequality']
image_file = PATHS['parameter_fit_results'] / Path('MLE', f'{measures[0]}_vs_{measures[1]}.png')
p.plot_vs(
    pair_measures=measures,
    file=image_file
)
measures = ['efficiency', 'attendance']
image_file = PATHS['parameter_fit_results'] / Path('MLE', f'{measures[0]}_vs_{measures[1]}.png')
p.plot_vs(
    pair_measures=measures,
    file=image_file
)

## All sizes

In [ ]:
file_all = Path(PATHS['parameter_fit_results'], 'MLE/best_fit_all.json')
df_all = pd.read_json(file_all)
df_all.index = df_all.model_name
df_parameters_all = df_all[['model_name', 'free_parameters']]
df_all.drop(columns=['model_name', 'free_parameters'], inplace=True)
df_all.sort_values(by='model_name', inplace=True)
df_all['deviance'] = df_all['deviance'].apply(lambda x: int(x))
df_all['AIC'] = df_all['AIC'].apply(lambda x: int(x))
df_all

In [ ]:
latex_string = df_all.to_latex()
print(latex_string)

In [ ]:
from Utils.LaTeX_utils import PrintLaTeX

latex_string_parameters = ''

for model, row in df_parameters_all.iterrows():
    if model in ['Attendance-M2']:
        parameters = row['free_parameters']
        latex_string_parameters += r'\textbf\{' + model + '\}\n'
        latex_string_parameters += PrintLaTeX.print_parameters(parameters)

print(latex_string_parameters)